In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import dask
import intake
import os
from collections import defaultdict
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
from xmip.utils import google_cmip_col
from xmip.postprocessing import combine_datasets, _match_datasets,_concat_sorted_time
import gcsfs
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

/tmp/ipykernel_1072/1491363843.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!


In [2]:
def reduce_cat_to_max_num_realizations(cmip6_cat):
    '''Reduce grid labels in pangeo cmip6 catalogue by 
    keeping grid_label and 'ipf' identifier combination with most datasets (=most realizations if using require_all_on)'''
    df = cmip6_cat.df
    cols = df.columns.tolist()
    
    df['ipf'] = [s[s.find('i'):] for s in df.member_id] #extract 'ipf' from 'ripf'

    #generate list of tuples of (source_id,ipf,grid_label) that provide most realizations (note this will omit realizations not available at this grid but possibly at others)
    max_num_ds_tuples = df.groupby(['source_id','ipf'])['grid_label'].value_counts().groupby(level=0).head(1).index.to_list() #head(1) gives (first) max. value since value_counts sorts max to min
    df_filter = pd.DataFrame(max_num_ds_tuples,columns=['source_id','ipf','grid_label']) #generate df to merge catalogue df on
    
    df = df_filter.merge(right=df, on = ['source_id','ipf','grid_label'], how='left') #do the subsetting
    df = df.drop(columns=['ipf']) #clean up
    df= df[cols]

    cmip6_cat.esmcat._df = df #(columns now ordered differently, probably not an issue?)
    return cmip6_cat

def drop_vars_from_cat(cmip6_cat,vars_to_drop):
    cmip6_cat.esmcat._df = cmip6_cat.df.drop(cmip6_cat.df[cmip6_cat.df.variable_id.isin(vars_to_drop)].index).reset_index(drop=True)
    return cmip6_cat

In [3]:
def preselect_years(ddict_in,start_year,end_year):
    '''select range of years of datasets'''
    ddict_out = defaultdict(dict)
    
    assert start_year<end_year
        
    if start_year>2014: #only using SSP
        for k, v in ddict_in.items():
            if 'ssp' in k:
                ddict_out[k] = v.sel(time=slice(str(start_year), str(end_year)))
                
    elif end_year<=2014: #only using historical
        for k, v in ddict_in.items():
            if 'historical' in k:
                ddict_out[k] = v.sel(time=slice(str(start_year), str(end_year)))
                
    elif ((start_year<=2014) & (end_year>2014)): #using both
        for k, v in ddict_in.items():
            if 'ssp' in k:
                ddict_out[k] = v.sel(time=slice('2015', str(end_year)))
            elif 'historical' in k:
                ddict_out[k] = v.sel(time=slice(str(start_year), '2014'))
    return ddict_out #NB: may result in no timesteps being selected at all

def drop_duplicate_timesteps(ddict_in):
    ddict_out = ddict_in
    for k, v in ddict_in.items():  
        unique_time, idx = np.unique(v.time,return_index=True)
        
        if len(v.time) != len(unique_time):
            ddict_out[k] = v.isel(time=idx)
            print('Dropping duplicate timesteps for:' + k)   
    return ddict_out

def drop_coords(ddict_in,coords_to_drop):
    for k, v in ddict_in.items():
        ddict_in[k] = v.drop_dims(coords_to_drop,errors="ignore")
    return ddict_in

def align_lonlat(ds_list):
    aligned_ds_list = []
    for ds in ds_list: #list of ds can't seem to be passed to xr.align instead
        a,b = xr.align(ds_list[0],ds,join='override',exclude=['time','member_id'])
        aligned_ds_list.append(b)
    return aligned_ds_list

def concat_members_aligning_lonlat(ds_list):
    aligned_ds_list = align_lonlat(ds_list) #override same-dimension lon/lat prior to concatenating (ensures lon/lats are not padded)
    return xr.concat(aligned_ds_list, dim='member_id', join='outer', coords='minimal',compat='override') #return xr.concat(ds_pick, dim='member_id')

def merge_variables_aligning_lonlat(ds_list):
    aligned_ds_list = align_lonlat(ds_list) #override same-dimension lon/lat prior to concatenating (ensures lon/lats are not padded)
    return xr.merge(aligned_ds_list, join='outer',compat='override')

Query runs & manipulate catalogue:

In [4]:
my_models = ['BCC-CSM2-MR','CESM2','CESM2-WACCM','CMCC-ESM2','CMCC-CM2-SR5',
                'GFDL-CM4','GFDL-ESM4','HadGEM3-GC31-MM','MIROC6','MPI-ESM1-2-HR','MRI-ESM2-0',
                'NorESM2-MM','TaiESM1']
'''
my_models = ['CESM2','CESM2-WACCM']
'''
col = google_cmip_col()

cat_data_ssp245 = col.search( #find instances providing all required variables for both historical & ssp245
    source_id=my_models,
    experiment_id=['historical','ssp245'],
    table_id='day',
    variable_id=['sfcWind','psl','pr'],
    require_all_on=['source_id', 'member_id','grid_label']
)


cat_data_ssp585 = col.search( #find instances providing all required variables for both historical & ssp585
    source_id=my_models,
    experiment_id=['historical','ssp585'],
    table_id='day',
    variable_id=['sfcWind','psl','pr'],
    require_all_on=['source_id', 'member_id','grid_label']
)
cat_data = cat_data_ssp585
cat_data.esmcat._df = pd.concat([cat_data_ssp245.df,cat_data_ssp585.df],ignore_index=True).drop_duplicates(ignore_index=True) #all instances we want

cat_data = reduce_cat_to_max_num_realizations(cat_data) #per model, select grid and 'ipf' combination providing most realizations
cat_data = drop_vars_from_cat(cat_data,['pr']) #we query only instances that also provide 'pr', but don't process 'pr' it in this script

In [5]:
cat_data.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NCAR,CESM2,historical,r11i1p1f1,day,psl,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190514
1,CMIP,NCAR,CESM2,historical,r11i1p1f1,day,sfcWind,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190514
2,ScenarioMIP,NCAR,CESM2,ssp245,r11i1p1f1,day,psl,gn,gs://cmip6/CMIP6/ScenarioMIP/NCAR/CESM2/ssp245...,NaN,20200528
3,ScenarioMIP,NCAR,CESM2,ssp245,r11i1p1f1,day,sfcWind,gn,gs://cmip6/CMIP6/ScenarioMIP/NCAR/CESM2/ssp245...,NaN,20200528
4,CMIP,NCAR,CESM2,historical,r4i1p1f1,day,sfcWind,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r4...,NaN,20190308
...,...,...,...,...,...,...,...,...,...,...,...
101,CMIP,AS-RCEC,TaiESM1,historical,r1i1p1f1,day,sfcWind,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20200626
102,ScenarioMIP,AS-RCEC,TaiESM1,ssp245,r1i1p1f1,day,psl,gn,gs://cmip6/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/s...,NaN,20210222
103,ScenarioMIP,AS-RCEC,TaiESM1,ssp245,r1i1p1f1,day,sfcWind,gn,gs://cmip6/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/s...,NaN,20210222
104,ScenarioMIP,AS-RCEC,TaiESM1,ssp585,r1i1p1f1,day,sfcWind,gn,gs://cmip6/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/s...,NaN,20200902


Open datasets into dictionary:

In [6]:
cat_data.esmcat.aggregation_control.groupby_attrs = [] #to circumvent aggregate=false bug

kwargs = {
    'zarr_kwargs':{
        'consolidated':True,
        'use_cftime':True
    },
    'aggregate':True #to avoid this issue: https://github.com/intake/intake-esm/issues/496
    #doesn't actually aggregate if we set cmip6_cat.esmcat.aggregation_control.groupby_attrs = []
}

ddict = cat_data.to_dataset_dict(**kwargs) #open datasets into dictionary

/tmp/ipykernel_1072/772692342.py:12: DeprecationWarning: cdf_kwargs and zarr_kwargs are deprecated and will be removed in a future version. Please use xarray_open_kwargs instead.
  ddict = cat_data.to_dataset_dict(**kwargs) #open datasets into dictionary



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


KeyboardInterrupt: 

In [ ]:
#ddict_manual = {k.split('/')[-1].replace('.zarr',''):xr.open_dataset(fs.get_mapper(k),engine='zarr',chunks={}) for k in fs.ls('leap-persistent/jbusecke/data/CMIP6/manual_test')}
ddict_manual = {k.split('/')[-1].replace('.zarr',''):xr.open_dataset(fs.get_mapper(k),engine='zarr',chunks={}) for k in fs.ls('leap-persistent/jbusecke/data/CMIP6/dataflow_test_production')}

#do preprocessing

#chunking structure changed, 100mb, smarter, if problems inform Julius

In [ ]:
list(ddict_manual.keys())

In [ ]:
#rename xmip possible
ddict = drop_duplicate_timesteps(ddict) #CESM2-WACCM has duplicate timeseries
ddict = preselect_years(ddict,1850,2100) #for now, limit analysis to up to 2100
ddict = drop_coords(ddict,['bnds','nbnd'])

Merge queried variables:

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}): #join=outer pads NaNs which result in large chunks for timeseries that differ in length
    ddict_merged = combine_datasets(ddict,merge_variables_aligning_lonlat,match_attrs=['source_id', 'grid_label', 'experiment_id', 'table_id','variant_label'])
    #ddict_concat = combine_datasets(ddict_merged,concat_members_aligning_lonlat,match_attrs=['source_id', 'grid_label', 'experiment_id', 'table_id'])

Do the regridding & subsetting. First, determine the grid coordinates around each tide gauge:

In [ ]:
mlrcoefs = xr.open_dataset('/home/jovyan/CMIP6cf/gssr_coefs_1degRes_forcing.nc') #contains coordinates of and MLR coefficients at TGs

era5_grid = xr.Dataset( #the ERA5 grid used to derive the MLR coefficients
        {   "longitude": (["longitude"], np.arange(-40,30,1)+1/2, {"units": "degrees_east"}),
            "latitude": (["latitude"], np.arange(70,10,-1)-1/2, {"units": "degrees_north"}),})

#get coordinates of n x n degree grids around each tide gauge
num_degr = 2
lat_ranges = np.zeros((len(mlrcoefs.tg),2))
lon_ranges = np.zeros((len(mlrcoefs.tg),2))

for t,tg in enumerate(mlrcoefs.tg.values):
    lat_ranges[t,:] = era5_grid.latitude[((era5_grid.latitude>=(mlrcoefs.sel(tg=tg).lat-num_degr/2)) & (era5_grid.latitude<=(mlrcoefs.sel(tg=tg).lat+num_degr/2)))][0:2]
    lon_ranges[t,:] = era5_grid.longitude[((era5_grid.longitude>=(mlrcoefs.sel(tg=tg).lon-num_degr/2)) & (era5_grid.longitude<=(mlrcoefs.sel(tg=tg).lon+num_degr/2)))][0:2]

#create da's to index the CMIP6 files with:
lons_da = xr.DataArray(lon_ranges,dims=['tg','lon_around_tg'],coords={'tg':mlrcoefs.tg,'lon_around_tg':[0,1]})
lats_da = xr.DataArray(lat_ranges,dims=['tg','lat_around_tg'],coords={'tg':mlrcoefs.tg,'lat_around_tg':[0,1]})

target_grid = era5_grid.sel(latitude=slice(np.max(lats_da)+2,np.min(lats_da)-2),longitude=slice(np.min(lons_da)-2,np.max(lons_da)+2))

Now, loop over each model/grid (should be only 1 grid per model), and apply the model/grid-specific interpolation weights to each dataset belonging to that model/grid

**Note:** It is probably more memory-efficient to loop over the tide gauges and regrid to each tide-gauge specific 2 by 2 degree grid, especially if considering larger domains. For now, we regrid to a subset of the ERA5 grid big enough to contain all tide-gauge specific grids (~20x20 grid points), and index the regridded dataset at the tide-gauge specific grids:

In [9]:
for key,ds in ddict_merged.items():
    lon_coord = list(k for k in ds.dims if 'lon' in k)[0] #find lon/lat coordinate names
    
    ds.coords[lon_coord] = ((ds.coords[lon_coord] + 180) % 360) - 180 #wrap around 0
    ds = ds.reindex({ lon_coord : np.sort(ds[lon_coord])})
    ddict_merged[key] = ds

ds_dict = {k: v for k, v in ddict_merged.items()}
ddict_subsetted = {k: v for k, v in ddict_merged.items()}

while len(ds_dict) > 0: #<- from xmip's combine_datasets
    k = list(ds_dict.keys())[0]
    ds = ds_dict.pop(k)
    ds.attrs["original_key"] = k
    
    matched_datasets = _match_datasets(ds, ds_dict, ['source_id', 'grid_label'], pop=True) #find datasets belonging to same model/grid
  
    regridder = xe.Regridder(matched_datasets[0],target_grid,'bilinear',ignore_degenerate=True) #interpolation weights for this grid
    #use periodic to avoid wrapping myself?
    
    for matched_ds in matched_datasets:
        first_ds,aligned_ds = xr.align(matched_datasets[0],matched_ds,join='override',exclude=['time','member_id','dcpp_init_year']) #make sure lon/lat coordinates are exactly the same
        
        aligned_ds = aligned_ds.isel(dcpp_init_year=0,drop=True) #get rid of dcpp_init_year dimension
        ddict_subsetted[matched_ds.original_key] = regridder(aligned_ds,keep_attrs=True).sel(latitude=lats_da,longitude=lons_da) #regrid to target grid and add to ddict

Store the datasets (directories structured per model):

In [7]:
for key,ds in tqdm(ddict_subsetted.items()):
    #model_path = os.path.join('/home/jovyan/CMIP6cf/output/subsetted_data/forcing_gssr_tgs/',ds.source_id)
    model_path = os.path.join('leap-persistent/timh37/CMIP6/subsetted_data/forcing_gssr_tgs/',ds.source_id)
    #if not os.path.exists(model_path):
    #    os.mkdir(model_path)
    #ds.to_netcdf(os.path.join(model_path,key.replace('.','_')+'.nc'),mode='w')
    ds.to_zarr(os.path.join('gs://',model_path,key.replace('.','_')+'.zarr'),mode='w')
    ds.close()

NameError: name 'ddict_subsetted' is not defined

^Takes about ~1,5h excluding EC-Earth3, 6h for EC-Earth3

Code for listing the instance_id of runs with missing timesteps:
```python
for k,v in ddict.items():
    if (len(v.time) < len(np.arange(1850,2015))*12*30):
    #if len(v.time) < len(np.arange(2015,2101))*12*30:
        print(v.attrs['intake_esm_attrs:zstore'][0:-1].replace('gs://cmip6/','').replace('/','.'))
```


    